#### 다음 실습 코드는 학습 목적으로만 사용 바랍니다. 문의 : audit@korea.ac.kr 임성열 Ph.D.

In [ ]:
%pip install langchain-core langchain-openai
%pip install langchain langchain-community
%pip install faiss-cpu

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 프롬프트 템플릿 설계
prompt = ChatPromptTemplate.from_template(
    "질문에 답변해주세요: {question}"
)

# 2. LLM 연결 모델 정의 (예: GPT-4o 사용) + API 키 포함
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.2,
    api_key=OPENAI_API_KEY
)

In [8]:
# 3. Chain 구성: 프롬프트 + LLM
chain: Runnable = prompt | llm
input_data = {"question": "Transformer 모델의 핵심 아이디어는 무엇인가요?"}

# 4. 체인 실행 및 결과 처리
response = chain.invoke(input_data)
print("응답:\n", response.content)

응답:
 Transformer 모델의 핵심 아이디어는 "어텐션 메커니즘"을 활용하여 입력 데이터의 모든 부분 간의 관계를 동시에 고려할 수 있도록 하는 것입니다. 이는 특히 자연어 처리(NLP) 작업에서 문맥을 이해하는 데 중요한 역할을 합니다. Transformer는 RNN이나 LSTM과 같은 순환 신경망과 달리 입력 시퀀스를 순차적으로 처리하지 않고, 병렬로 처리할 수 있어 학습 속도가 빠르고, 긴 문맥을 효과적으로 처리할 수 있습니다.

Transformer 모델은 주로 두 가지 구성 요소로 이루어져 있습니다: 인코더와 디코더. 인코더는 입력 시퀀스를 처리하여 의미 있는 표현을 생성하고, 디코더는 이 표현을 기반으로 원하는 출력 시퀀스를 생성합니다. 이 과정에서 "셀프 어텐션" 메커니즘을 사용하여 각 단어가 문맥 내 다른 단어들과 어떻게 관련되어 있는지를 계산합니다. 이를 통해 모델은 문장의 구조와 의미를 더 잘 이해할 수 있게 됩니다.


In [10]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# 문서 준비 및 텍스트 분할
documents = [
    Document(page_content="Transformers use self-attention to process tokens in parallel."),
    Document(page_content="The attention mechanism allows the model to focus on relevant parts of the input."),
    Document(page_content="Positional encoding is used to retain order information in Transformers."),
]

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
docs = splitter.split_documents(documents)

In [11]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate

# 1. 벡터 DB 연결
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectorstore = FAISS.from_documents(docs, embeddings)

# 2. 검색기 (Retriever) 구성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

# 3. 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_template(
    "다음 문맥을 참고하여 질문에 답하세요.\n\n문맥:\n{context}\n\n질문:\n{question}"
)

# 4. LLM 모델 정의 (GPT-4o)
model = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    api_key=OPENAI_API_KEY
)

In [12]:
# RAG 체인 구현 LCEL 기반
rag_chain = (
    {        
     "context": retriever,
     "question": RunnablePassthrough()
     }
    | prompt
    | model
    | StrOutputParser()
    )

# 질문 예시 및 실행
question = "Transformer에서 self-attention은 왜 중요한가요?"
answer = rag_chain.invoke(question)

print("질문:", question)
print("답변:", answer)


질문: Transformer에서 self-attention은 왜 중요한가요?
답변: Transformer에서 self-attention은 중요한 이유는 여러 가지가 있습니다. 첫째, self-attention 메커니즘은 입력 토큰들을 병렬로 처리할 수 있게 해주어 모델의 효율성을 높입니다. 둘째, self-attention은 입력 데이터의 중요한 부분에 집중할 수 있도록 도와줍니다. 이는 모델이 입력의 특정 부분에 더 많은 가중치를 부여하여 더 나은 성능을 발휘할 수 있게 합니다. 이러한 특성 덕분에 self-attention은 Transformer 모델의 핵심 요소로 자리잡고 있습니다.
